In [1]:
pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=6df070eb574e63178208f5c52ef550600c6816a07d7d42ad238b5f2f20aee1cd
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [2]:
pip install ultralytics==8.0.196

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 12.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [3]:
pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0


In [4]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [5]:
import numpy as np
import cv2
import os
import argparse
from utils import *
from roboflow import Roboflow
from ultralytics import YOLO
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans


In [6]:
from roboflow import Roboflow
rf = Roboflow(api_key="5AYKbygCi5IOpsIwX71T")
project = rf.workspace("new-amina").project("object-detection-new-amina-dina")
version = project.version(17)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to object-detection-new-amina-dina-17 in yolov8:: 100%|██████████| 1920/1920 [00:01<00:00, 1887.79it/s]


In [7]:
# Define paths
model_path = '/content/best .pt'  # Path to your YOLOv8 model
input_dir = '/content/object-detection-new-amina-dina-17/train/images'  # Path to the input directory with images
output_dir = '/content/output'  # Path to the output directory to save results

# Load the YOLOv8 model
model = YOLO(model_path)

# Ensure the output directory exists
Path(output_dir).mkdir(parents=True, exist_ok=True)

def lane_detection(image, road_bbox):
    x1, y1, x2, y2 = road_bbox
    road_region = image[y1:y2, x1:x2].copy()

    # Convert to grayscale
    gray = cv2.cvtColor(road_region, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian Blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    # Edge detection using Sobel operator
    sobelx = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=5)
    edges = cv2.magnitude(sobelx, sobely)

    # Convert edges to binary
    _, edges = cv2.threshold(edges, 50, 255, cv2.THRESH_BINARY)

    # Extract edge points for clustering
    points = np.column_stack(np.where(edges > 0))

    if len(points) > 0:
        # Split points into two clusters
        kmeans = KMeans(n_clusters=2).fit(points)
        labels = kmeans.labels_

        lane_mask = np.zeros_like(road_region)

        left_points = points[labels == 0]
        right_points = points[labels == 1]

        # Fit linear regression models to each set of points
        left_reg = LinearRegression()
        right_reg = LinearRegression()
        if len(left_points) > 0:
            left_reg.fit(left_points[:, 0].reshape(-1, 1), left_points[:, 1])
        if len(right_points) > 0:
            right_reg.fit(right_points[:, 0].reshape(-1, 1), right_points[:, 1])

        # Generate the fitted line points
        left_fitted_line = left_reg.predict(np.arange(road_region.shape[0]).reshape(-1, 1))
        right_fitted_line = right_reg.predict(np.arange(road_region.shape[0]).reshape(-1, 1))

        # Create a polygon from the fitted lines
        left_lane_points = np.column_stack((left_fitted_line, np.arange(road_region.shape[0])))
        right_lane_points = np.column_stack((right_fitted_line, np.arange(road_region.shape[0])))
        right_lane_points = np.flip(right_lane_points, axis=0)
        lane_points = np.vstack((left_lane_points, right_lane_points))

        # Fill the area between the lanes
        cv2.fillPoly(lane_mask, [lane_points.astype(np.int32)], (0, 0, 255))

        result = cv2.addWeighted(road_region, 1, lane_mask, 0.5, 0)
        image[y1:y2, x1:x2] = result

    return image

input_images = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

for img_name in input_images:
    img_path = os.path.join(input_dir, img_name)
    img = cv2.imread(img_path)

    # YOLO object detection
    results = model(img)

    road_bbox = None
    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, confidence, label = box[:6]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            confidence = float(confidence)
            label = int(label)

            # Filtering 'road' class has label 8
            if label == 9:
                road_bbox = (x1, y1, x2, y2)
                break

        if road_bbox:
            break

    if road_bbox:
        # Draw the bounding box for debugging
        cv2.rectangle(img, (road_bbox[0], road_bbox[1]), (road_bbox[2], road_bbox[3]), (0, 255, 0), 2)
        img_with_lanes = lane_detection(img.copy(), road_bbox)
    else:
        img_with_lanes = img

    # Save the final image
    output_img_path = os.path.join(output_dir, img_name)
    cv2.imwrite(output_img_path, img_with_lanes)

print('==> All done!')
print('***********************************************************')



WARNING ⚠️ NMS time limit 0.550s exceeded
0: 800x800 7 bicycles, 3 buildings, 9 cars, 1 myCar, 15 persons, 1 pole, 1 road, 2 sidewalks, 1 sky, 1 terrain, 9 traffic-lights, 11 traffic-signs, 4 vegetations, 49.6ms
Speed: 23.7ms preprocess, 49.6ms inference, 1283.2ms postprocess per image at shape (1, 3, 800, 800)
The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

0: 800x800 3 bicycles, 4 buildings, 12 cars, 1 myCar, 4 poles, 1 road, 2 sidewalks, 1 terrain, 11 traffic-signs, 3 vegetations, 22.7ms
Speed: 5.8ms preprocess, 22.7ms inference, 2.2ms postprocess per image at shape (1, 3, 800, 800)
The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

0: 800x800 6 bicycles, 2 buildings, 11 cars, 1 myCar, 1 pole, 3 riders, 1 road, 2 sidewalks, 1 sky, 1 terrain, 5 traffic-lights, 2 traffic-signs, 4 vegetations, 22.6ms
Speed: 5.1ms preprocess, 2

==> All done!
***********************************************************
